In [1]:
import pandas as pd
import os 
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import japanize_matplotlib
import openpyxl

In [2]:
base_path = os.path.dirname(os.path.abspath("")) #本番 (__file__)
questionnaire_contract = os.path.normpath(os.path.join(base_path, "test/契約_回答.xlsx"))
questionnaire_claim = os.path.normpath(os.path.join(base_path, "test/損害_回答.xlsx"))

In [3]:
contract_file = pd.read_excel(questionnaire_contract, engine='openpyxl',index_col=0,header=2)
claim_file = pd.read_excel(questionnaire_claim, engine='openpyxl',index_col=0,header=2)

C:\Users\esple\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\esple\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
#推奨度抽出
contract_file = contract_file.copy()
claim_file = claim_file.copy()

def recommendData(file):
    copy_df = file.copy()
    index_reset = copy_df.reset_index().set_index("送信日")
    def extractRecommendData(data,agentNum):
        recommend_column = data.loc[:,"推奨度"]
        recommend_mean = recommend_column.resample("ME").mean("推奨度").rename("平均値_" + agentNum).round(2)
        recommend_count = recommend_column.resample("ME").count().rename("個数_" + agentNum)
        recommend_df = pd.concat([recommend_mean,recommend_count], axis=1)
        return recommend_df
    def agentExtractRecommendData(data,agent_list):
        _df = pd.DataFrame()
        for i in agent_list:
            agent_detaset = data[data.loc[:,"代理店コード"] == i]
            agent_df = extractRecommendData(agent_detaset,str(i))
            _df = pd.concat([_df,agent_df], axis=1)
        return _df
    recommend_df_all = extractRecommendData(index_reset,"0702")
    agent_num_list = index_reset.iloc[:,5].unique()
    recommend_df_agent = agentExtractRecommendData(index_reset,agent_num_list)
    recommend_df = pd.concat([recommend_df_all,recommend_df_agent],axis=1)
    fillna_0 = recommend_df.fillna(0)
    return fillna_0

claim_recommend = recommendData(claim_file)
contract_recommend = recommendData(contract_file)
contract_recommend

,平均値_0702,個数_0702,平均値_862,個数_862,平均値_885,個数_885,平均値_795,個数_795,平均値_765,個数_765,...,平均値_769,個数_769,平均値_2310,個数_2310,平均値_761,個数_761,平均値_865,個数_865,平均値_767,個数_767
送信日,,,,,,,,,,,,,,,,,,,,,
2023-07-31,7.56,9,7.00,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,10.0,1.0,3.00,1.0,0.0,0.0,0.0,0.0
2023-08-31,8.38,8,8.00,3,0.0,0.0,0.0,0.0,10.0,1.0,...,8.00,1.0,0.0,0.0,8.00,1.0,0.0,0.0,0.0,0.0
2023-09-30,7.20,10,7.25,4,0.0,0.0,0.0,0.0,0.0,0.0,...,7.50,2.0,0.0,0.0,7.67,3.0,0.0,0.0,5.0,1.0
2023-10-31,8.00,12,7.00,2,0.0,0.0,0.0,0.0,0.0,0.0,...,9.33,3.0,0.0,0.0,9.00,2.0,0.0,0.0,0.0,0.0
2023-11-30,7.00,4,5.00,1,0.0,0.0,0.0,0.0,0.0,0.0,...,5.00,1.0,0.0,0.0,8.00,1.0,10.0,1.0,0.0,0.0
2023-12-31,7.67,9,7.60,5,0.0,0.0,0.0,0.0,6.5,2.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2024-01-31,8.11,9,8.33,3,0.0,0.0,0.0,0.0,7.5,2.0,...,9.00,2.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2024-02-29,7.20,20,7.50,8,5.0,1.0,0.0,0.0,8.5,2.0,...,5.00,2.0,0.0,0.0,7.50,4.0,4.0,1.0,0.0,0.0
2024-03-31,6.67,18,7.50,4,4.0,2.0,5.0,1.0,6.0,2.0,...,8.00,1.0,8.0,2.0,5.00,1.0,0.0,0.0,0.0,0.0


In [18]:

# entire_contract_recommend = contract_recommend.iloc[:,[0,1]]
# entire_claim_recommend = claim_recommend.iloc[:,[0,1]]
# contract_recommend
data_name_list = ["グラフ_契約","グラフ_損害"]
def add_gpaph_recommend(data,data_name):
    _df = data.iloc[:,[0,1]]
    if data_name == data_name_list[0]:
        list_number = 0
    else:
        list_number = 1
    graph_name = ["推奨度_契約","推奨度_損害"]

    _df.plot(
        subplots=True,
        title=graph_name[list_number],
        grid=True,
        colormap='Set1',
        legend=True,
        alpha=1.0,
        figsize=(8, 6),
        sharex=False,
        marker="o"
    )
    image_file_path = base_path + "/test/tmp.png"
    plt.savefig(image_file_path)
    img = openpyxl.drawing.image.Image(image_file_path)
    plt.close('all')
    # エクセル出力
    current_month = str(datetime.now().month)
    current_year = str(datetime.now().year)
    excel_new_path = os.path.normpath(os.path.join(base_path + "/test/" + current_year + "_" + current_month + '_アンケート集計.xlsx'))

    wb = openpyxl.load_workbook(excel_new_path)
    sh = wb.create_sheet(data_name)
    sh.add_image(img, "B2")
    wb.save(excel_new_path)
    wb.close()
    
add_gpaph_recommend(contract_recommend,data_name_list[0])
add_gpaph_recommend(claim_recommend,data_name_list[1])
